In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from pandas.tseries.offsets import DateOffset

In [7]:
dataset = pd.read_csv('inn.csv')
dropped = ['Unnamed: 0','total_cases','total_deaths','new_deaths','new_tests','population','population_density','median_age','gdp_per_capita',
           'extreme_poverty','cardiovasc_death_rate','diabetes_prevalence','life_expectancy',
           'day','month','year','quarter','half','bimonth', 'cod_continent','prev_cases','prev_deaths','cod_location']
dataset = dataset[dataset['cod_location'] == 52]
dataset.drop(dropped,inplace=True,axis=1)

In [8]:
dataset.columns
dataset.set_index('date_id',inplace=True) 

In [5]:
train, test = dataset[:-60], dataset[-60:]

In [6]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

ValueError: could not convert string to float: '2020-03-11'

In [ ]:
n_input = 60
n_features = 2
generator = TimeseriesGenerator(train,train,length = n_input,batch_size = 6)
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit_generator(generator, epochs= 180)

In [ ]:
pred_list = []
batch = train[-n_input].reshape((1,n_input,n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:], [[pred_list[i]]], axis = 1)
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=dataset[-n_input:].index, columns=['prediction'])
df_test = pd.concat([dataset,df_predict],axis = 1)
plt.figure(figsize=(20,5))
plt.plot(df_test.index,df_test['new_cases'])
plt.plot(df_test.index,df_test['prediction'],color ='r')
plt.show()

In [ ]:
train = dataset
scaler.fit(train)
train = scaler.transform(train)
n_input = 30
n_features = 2
generator = TimeseriesGenerator(train,train,length = n_input,batch_size = 6)

model.fit_generator(generator, epochs= 180)

pred_list = []
batch = train[-n_input].reshape((1,n_input,n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:], [[pred_list[i]]], axis = 1)
add_dates = [dataset.index[-1] + DateOffset(months = x) for x in range(0,13)]
future_dates = dataset.DataFrame(index=add_dates[1:],columns=dataset.columns)
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                         index=future_dates[-n_input:].index,columns=['prediction'])
df_proj = pd.concat([dataset,df_predict], axis = 1)

plt.figure(figsize=(10,4))
plt.plot(df_test.index,df_test['total_cases'])
plt.plot(df_test.index,df_test['prediction'],color ='r')
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()